In [ ]:
# D:\Desktop\Final Year Project\DataSets\Celeb-DF\Celeb-real

In [1]:
import glob
import cv2
import numpy as np
from tqdm import tqdm
from facenet_pytorch import MTCNN
import os
import json

In [3]:
# video_files =  glob.glob('D:/Desktop/Final Year Project/DataSets/Celeb-DF/Celeb-real/*mp4')
video_files =  sorted(glob.glob('D:/Desktop/Final Year Project/DataSets/Celeb-DF-v2/Celeb-synthesis/*mp4'))
len(video_files)

5639

In [4]:
# Counting frames of each video and remove video if frame size < 150
frame_count = []
for video_file in tqdm(video_files):
  cap = cv2.VideoCapture(video_file)
  if(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))<150):
    video_files.remove(video_file)
    continue
  frame_count.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
  
print("frames : " , frame_count)
print("Total number of videos : " , len(frame_count))
print('Average frame per video :',np.mean(frame_count))

  0%|          | 0/5639 [00:00<?, ?it/s]

100%|█████████▉| 5635/5639 [02:08<00:00, 43.91it/s]


frames :  [469, 303, 350, 529, 326, 459, 534, 479, 464, 520, 469, 303, 350, 529, 459, 534, 479, 520, 469, 303, 350, 529, 459, 534, 479, 520, 469, 303, 350, 529, 326, 459, 534, 479, 520, 469, 303, 350, 529, 326, 459, 534, 479, 520, 469, 303, 350, 326, 459, 479, 520, 469, 303, 350, 529, 326, 459, 534, 479, 464, 520, 469, 303, 350, 529, 326, 459, 534, 479, 464, 520, 469, 303, 350, 529, 326, 459, 534, 479, 464, 520, 469, 303, 350, 529, 326, 459, 534, 479, 464, 520, 469, 303, 350, 529, 326, 459, 534, 520, 469, 303, 350, 534, 479, 520, 469, 303, 350, 529, 459, 534, 479, 464, 520, 343, 517, 318, 161, 496, 380, 343, 517, 318, 161, 496, 380, 343, 517, 318, 161, 496, 380, 343, 517, 318, 161, 496, 380, 535, 349, 441, 357, 313, 337, 327, 316, 320, 371, 397, 330, 398, 364, 339, 314, 308, 376, 398, 364, 339, 314, 308, 327, 349, 340, 352, 309, 452, 459, 466, 510, 390, 489, 327, 349, 400, 340, 352, 309, 452, 459, 435, 466, 510, 489, 317, 316, 363, 303, 410, 345, 224, 284, 315, 325, 272, 308, 311, 317,

In [5]:
# Extract frames from video
def frame_extract(path):
    vidObj = cv2.VideoCapture(path) 
    success = True
    while success:
        success, image = vidObj.read()
        if success:
            yield image

In [6]:
# Process and save videos with faces only using MTCNN

def create_face_videos(path_list, out_dir):
    detector = MTCNN()
    os.makedirs(out_dir, exist_ok=True)

    # Filter out already processed videos
    processed = set(os.path.basename(f) for f in glob.glob(os.path.join(out_dir, '*.mp4')))
    path_list = [p for p in path_list if os.path.basename(p) not in processed]

    print("Total videos to process:", len(path_list))
    print("Already processed videos:", len(processed))

    for path in tqdm(path_list, desc="Processing Videos"):
        filename = os.path.basename(path)
        out_path = os.path.join(out_dir, filename)

        out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*'MJPG'), 30, (112, 112))
        idx = 0
        for frame in frame_extract(path):
            if idx > 150:
                break

            result = detector.detect_faces(frame)
            if result:
                x, y, w, h = result[0]['box']
                x, y = max(0, x), max(0, y)
                face_img = frame[y:y+h, x:x+w]
                if face_img.size != 0:
                    face_resized = cv2.resize(face_img, (112, 112))
                    out.write(face_resized)
            idx += 1

        out.release()

In [7]:
create_face_videos(video_files,'D:/Desktop/Final Year Project/DataSets/data_preprocessing/celeb_df/fake_face_only/')

Total videos to process: 0
Already processed videos: 5635


Processing Videos: 0it [00:00, ?it/s]


In [ ]:
# Upscaling function
def upscale_video(input_path, output_path, target_size=(224, 224)):
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print(f"Failed to open: {input_path}")
        return

    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  
    out = cv2.VideoWriter(output_path, fourcc, fps, target_size)

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        resized = cv2.resize(frame, target_size, interpolation=cv2.INTER_CUBIC)
        out.write(resized)

    cap.release()
    out.release()


input_dir = "D:/Desktop/Final Year Project/DataSets/data_preprocessing/celeb_df/fake_face_only"   
output_dir = "D:/Desktop/Final Year Project/DataSets/data_preprocessing/celeb_df/fake_face_only224"  

os.makedirs(output_dir, exist_ok=True)

video_files = [f for f in os.listdir(input_dir) if f.endswith(".mp4")]

for file in tqdm(video_files, desc="🔼 Upscaling"):
    in_path = os.path.join(input_dir, file)
    out_path = os.path.join(output_dir, file)
    upscale_video(in_path, out_path)
